In [ ]:
import numpy as np
import xfab.tools
import os
import numpy as np
import matplotlib.pyplot as plt
from xfab.symmetry import Umis
import xfab.symmetry
from collections import deque
from scipy.ndimage import binary_fill_holes
from scipy.spatial.transform import Rotation
import skimage


In [ ]:
def get_u_map(ubi_map, sample_mask):
    u_map = np.zeros_like(ubi_map)
    for i in range(ubi_map.shape[0]):
        for j in range(ubi_map.shape[1]):
            if sample_mask[i,j]:
                ubi = ubi_map[i,j]
                u = xfab.tools.ubi_to_u(ubi)
                u_map[i,j] = u
    return u_map

def kernel_average_misorientation(orientation_map,
                                  footprint,
                                  crystal_system,
                                  misorientation_threshold=(0, np.inf),
                                  mask=None,
                                  fill_value=np.nan):
    """Apply a kernel average misorientation (KAM) filter to the input orientation map.

    The KAM filter is designed as described here: https://mtex-toolbox.github.io/EBSDKAM.html

    Args:
        orientation_map (:obj:`numpy array`): the pixelated orientation matrix field, shape=(M, N, 3, 3).
        footprint (:obj:`numpy array`): boolean array defining the kenrel neighbourhood, shape=(m, n).
        crystal_system (:obj:int): crystal_system number must be one of 1: Triclinic, 2: Monoclinic,
            3: Orthorhombic, 4: Tetragonal, 5: Trigonal, 6: Hexagonal, 7: Cubic
        misorientation_threshold (tuple of :obj:`float`): Reject misorientations outside specified range. 
            Defaults to (0, np.inf).
        mask (:obj:`numpy array`): Boolean array, where to skipp pixels, shape=(M, N). Defaults to None.
        fill_value (:obj:`numpy array`): To put where mask is false. Defaults to np.nan.

    Returns:
        :obj:`numpy array`: the scalar kernel average misorientation map, shape=(M, N).
    """
    assert footprint.shape[0]%2!=0
    assert footprint.shape[1]%2!=0

    m = footprint.shape[0] // 2
    n = footprint.shape[1] // 2
    kam_map = np.zeros((orientation_map.shape[0], orientation_map.shape[1]))
    lower_bound, upper_bound = misorientation_threshold
    for i in range(m, orientation_map.shape[0]-m ):
        if i%10==0: print(str( np.round(i / float(orientation_map.shape[0])) ), end='% , ')
        for j in range(n, orientation_map.shape[0]-n):
            if mask is not None and mask[i,j]:
                U = orientation_map[i, j]
                local_average_misorientation = 0
                number_of_pixels = 0
                for k in range(footprint.shape[0]):
                    for l in range(footprint.shape[1]):
                        if footprint[k, l]>0:
                            row = i - m + k
                            col = j - n + l
                            if mask is not None and mask[row, col]:
                                u = orientation_map[row, col]
                                misorientation = Umis( U, u, crystal_system )[:,1].min()
                                if misorientation < upper_bound and misorientation > lower_bound:
                                    local_average_misorientation += misorientation
                                    number_of_pixels += 1
                if number_of_pixels>0:
                    kam_map[i, j] += (local_average_misorientation / number_of_pixels)
            else:
                kam_map[i, j] = fill_value
    return kam_map

def flood_fill( orientation_map, 
                seed_point, 
                footprint, 
                crystal_system, 
                local_disorientation_tolerance,
                global_disorientation_tolerance, 
                mask=None, 
                background_value=np.nan,
                fill_holes=False,
                max_grains = 99,
                min_grain_size = 0,
                verbose=False
                ):
    assert footprint.shape[0]%2!=0
    assert footprint.shape[1]%2!=0

    M, N = orientation_map.shape[0], orientation_map.shape[1]
    segmentation = np.zeros((M,N))
    skipps = np.ones((M,N), dtype=bool)
    label = 1
    segmentation[seed_point[0], seed_point[1]:seed_point[1] + 3] = label
    done = False
    iteration = 0
    while( not done and iteration < max_grains ):
        rows, cols = np.where( mask*(segmentation==0)*skipps )
        if len(rows)>0:
            n = np.random.randint( 0, len(rows) )
            seed_point = ( rows[n], cols[n] )
            grain_mask = _flood(orientation_map, 
                                seed_point, 
                                footprint, 
                                crystal_system, 
                                local_disorientation_tolerance,
                                global_disorientation_tolerance,
                                mask)
            if fill_holes: grain_mask = binary_fill_holes(grain_mask)
            if np.sum(grain_mask) > min_grain_size:
                segmentation[grain_mask] = label
                label += 1
            else:
                skipps[grain_mask] = False
            iteration += 1
            if verbose:
                print('Iteration ', iteration, ', found grain with : ', np.sum(grain_mask), ' voxels @, seed_point, ', np.sum(grain_mask))
        else:
            done = True

    segmentation[segmentation==0] = background_value
    return segmentation

def _align( us, crystal_system ):
    """Align a set of orientation matrices as closely as possible given a lattice symmetry.
    """
    rot = xfab.symmetry.rotations(crystal_system)
    ref = us[0]
    newu = np.zeros_like(us)
    for i,u in enumerate(us):
        mis = xfab.symmetry.Umis( u, ref, crystal_system )
        rotindex = np.argmin(mis[:,1])
        newu[i,:,:] =  u @ rot[rotindex]
    return newu


def _flood( orientation_map, 
            seed_point, 
            footprint, 
            crystal_system,
            local_disorientation_tolerance,
            global_disorientation_tolerance,
            mask ):
    m = footprint.shape[0] // 2
    n = footprint.shape[1] // 2
    flood_mask = np.zeros((orientation_map.shape[0], orientation_map.shape[1]), dtype=bool)

    i, j = seed_point
    flood_mask[i, j] = True
    if mask is not None and mask[i, j]==0: raise ValueError('Seed point not in mask')
    
    unchartered_indices = deque([ seed_point ])
    while(  len(unchartered_indices) > 0 ):
        i, j = unchartered_indices.pop()
        U = orientation_map[i, j]
        
        if np.sum(flood_mask) < 20:
            us = _align( orientation_map[flood_mask,:,:], crystal_system )
            global_U = Rotation.from_matrix( us ).mean().as_matrix()

        for k in range(footprint.shape[0]):
            for l in range(footprint.shape[1]):
                if footprint[k, l]>0:
                    row = i - m + k
                    col = j - n + l
                    if not flood_mask[row, col] and mask is not None and mask[row, col]:
                        u = orientation_map[row, col]
                        misorientation = Umis( U, u, crystal_system )[:,1].min()
                        
                        if misorientation < local_disorientation_tolerance:
                            glob_misorientation = Umis( global_U, u, crystal_system )[:,1].min()
                            if glob_misorientation < global_disorientation_tolerance:
                                flood_mask[row, col] = True
                                unchartered_indices.appendleft(  ( row, col) )
    return flood_mask


In [ ]:
keys = ['layer_top', 'layer_center', 'layer_bottom']
dirs = [os.path.join('..', dir, 'test_run') for dir in keys]
sample = {}
sample['layer_top']    = np.fliplr(  np.load(os.path.join(dirs[0], 'sample_mask_layer_top.npy'))[5:-5, 5:-5]  )
sample['layer_center'] = np.fliplr(  np.load(os.path.join(dirs[1], 'sample_mask_layer_center.npy'))[2:-2, 2:-2] )
sample['layer_bottom'] = np.fliplr(  np.load(os.path.join(dirs[2], 'sample_mask_layer_bottom.npy'))[2:-2, 2:-2] )

ubi_map, X, Y, ipf_map = {},{},{},{}
for i,key in enumerate(keys):
    X[key] = np.load( os.path.join( dirs[i], 'X_coord_gmap.npy' ) )
    Y[key] = np.load( os.path.join( dirs[i], 'Y_coord_gmap.npy' ) )
    ubi_map[key]    = np.load(os.path.join(dirs[i], 'ubi_median_filt_s6.npy'))
    ipf_map[key]    = np.load(os.path.join(dirs[i], 'ipf_median_filt_s6.npy'))



In [ ]:
if 0:

    footprint= np.array([[ 0, 0 ,1, 0, 0 ],
                            [ 0, 1, 1, 1, 0 ],
                            [ 1, 1, 1, 1, 1 ],
                            [ 0, 1, 1, 1, 0 ],
                            [ 0, 0, 1, 0, 0 ]])

    for key in keys:
        print(key)
        print(' ')
        orientation_map = get_u_map(ubi_map[key], sample[key])

        kam = kernel_average_misorientation( orientation_map,
                                            footprint=footprint,
                                            crystal_system=7,
                                            misorientation_threshold=(1.8, np.inf),
                                            mask = sample[key],
                                            fill_value=np.nan )
        np.save(key + '_kam.npy', kam)
        print(' ')


In [ ]:
kam = {}
for i,key in enumerate(keys):
    kam[key] = np.load(key + '_kam.npy')

In [ ]:
grain_boundaries = { }
for key in keys:
    field = kam[key].copy( )
    field[field < 4.0] = 0
    field[field > 4.0] = 1
    field = field.astype(bool) * sample[key]
    bound = ~skimage.morphology.binary_erosion(sample[key]) * sample[key]
    field[bound] = True
    field = skimage.morphology.thin(field, max_num_iter=None) * sample[key]
    field = skimage.morphology.remove_small_objects(field, min_size=64, connectivity=2)
    grain_boundaries[key] = field                            
    if 0: np.save(key + '_grain_boundaries.npy', field)



In [ ]:
for key in keys:
    fig, ax = plt.subplots(1,1,figsize=(9,9))
    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    cropped = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()
    ipf = ipf_map[key].copy()
    ipf[~sample[key]] = np.nan
    ipf = ipf[x_min:x_max+1, y_min:y_max+1].copy()
    ipf[cropped] = 0
    Xc = X[key][x_min:x_max+1, y_min:y_max+1]
    Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

    im = ax.pcolormesh( Xc, Yc, ipf[:,:,:,2])

    ax.axis('off')


    ax.axis('equal')

    if 0: fig.savefig('gallery/' + key +'_ipf_Y_with_GB.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
    plt.show()

In [ ]:
for key in keys:
    kam[key][sample[key]==0] = np.nan

    fig, ax = plt.subplots(1,1,figsize=(9,9))
    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    cropped = kam[key][x_min:x_max+1, y_min:y_max+1]
    cropped[cropped.sum(axis=-1)==0]=np.nan

    Xc = X[key][x_min:x_max+1, y_min:y_max+1]
    YC = Y[key][x_min:x_max+1, y_min:y_max+1]
    im = ax.imshow(  np.rot90(cropped), cmap='Paired', vmin=0, vmax=60)

    ax.axis('off')

    ax.axis('equal')

    if 0: fig.savefig('gallery/' + key +'_kam.png', dpi=600, pad_inches=0.000001, bbox_inches='tight')

    plt.show()

In [ ]:
colors = np.array( [
    [55,  126, 184],  #377eb8 
    [255, 127, 0],    #ff7f00
    [77,  175, 74],   #4daf4a
    [247, 129, 191],  #f781bf
    [166, 86,  40],   #a65628
    [152, 78,  163],  #984ea3
    [153, 153, 153],  #999999
    [228, 26,  28],   #e41a1c
    [222, 222, 0]     #dede00
] )
colors = colors / np.max(colors, axis=1).reshape(9,1)

for i, key in enumerate(keys):
    fig, ax = plt.subplots(1,1,figsize=(12,12))

    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)

    gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

    Xc = X[key][x_min:x_max+1, y_min:y_max+1]
    Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

    mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]

    rgb_image = np.zeros((*Xc.shape, 4))
    rgb_image[gb, -1]  = 1
    rgb_image[gb, 0:3] = 0

    im = ax.pcolormesh( Xc, Yc, rgb_image, label=key.replace('_',' ') )

    ax.axis('off')

    if 1: fig.savefig('gallery/gbs_'+key+'.png', dpi=600, pad_inches=0.000001, bbox_inches='tight')
plt.show()

In [ ]:
colors = np.array( [
    [55,  126, 184],  #377eb8 
    [255, 127, 0],    #ff7f00
    [77,  175, 74],   #4daf4a
    [247, 129, 191],  #f781bf
    [166, 86,  40],   #a65628
    [152, 78,  163],  #984ea3
    [153, 153, 153],  #999999
    [228, 26,  28],   #e41a1c
    [222, 222, 0]     #dede00
] )
colors = colors / np.max(colors, axis=1).reshape(9,1)

fig, ax = plt.subplots(1,1,figsize=(12,12))
for i, key in enumerate(keys):

    # blue = top
    # orange = centre
    # green  == bottom
    coords = np.argwhere(sample[key])
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)

    gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

    Xc = X[key][x_min:x_max+1, y_min:y_max+1]
    Yc = Y[key][x_min:x_max+1, y_min:y_max+1] 

    mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]

    rgb_image = np.zeros((*Xc.shape, 4))
    rgb_image[gb, -1]  = 1
    rgb_image[gb, 0:3] = colors[i]

    im = ax.pcolormesh( Xc, Yc, rgb_image, label=key.replace('_',' ') )

ax.axis('off')

if 0: fig.savefig('gallery/gb_error.png', dpi=600, pad_inches=0.000001, bbox_inches='tight')
plt.show()

In [ ]:


x0 = X[keys[0]][grain_boundaries[keys[0]]]
y0 = Y[keys[0]][grain_boundaries[keys[0]]]

x1 = X[keys[1]][grain_boundaries[keys[1]]]
y1 = Y[keys[1]][grain_boundaries[keys[1]]]

x2 = X[keys[2]][grain_boundaries[keys[2]]]
y2 = Y[keys[2]][grain_boundaries[keys[2]]]

dist = [np.sqrt( np.min( (x - x1)**2 + (y - y1)**2 ) ) for x,y in zip(x0,y0)]
dist.extend( [np.sqrt( np.min( (x - x1)**2 + (y - y1)**2 ) ) for x,y in zip(x2,y2)] )


In [ ]:
y_x,y_y = [],[]
for x,y in zip(x0,y0):
    d = np.sqrt(  (x - x1)**2 + (y - y1)**2 )
    index = np.argmin( d )
    y_x.append( x - x1[index] )
    y_y.append( y - y1[index] )
for x,y in zip(x2,y2):
    d = np.sqrt( (x - x1)**2 + (y - y1)**2  )
    index = np.argmin( d )
    y_x.append( x - x1[index] )
    y_y.append( y - y1[index] )
y = np.array([y_x,y_y])

In [ ]:
maxd = np.linalg.norm(y,axis=0).max()
Delta_z = 3
alpha_max = np.arctan(Delta_z / maxd)
print('alpha_max', np.degrees(alpha_max), 'degrees')
print('Delta_z', Delta_z, 'microns')

In [ ]:
sample_size = int( 1e7 )
white_noise = np.random.normal(size=(3, sample_size))
nhat = white_noise / np.linalg.norm(white_noise, axis=0)
m = np.arccos(np.abs(nhat[2,:]))  > alpha_max
nhat = nhat[:,m]
nx, ny, nz = nhat
ux = -(Delta_z/ ((1-nz*nz)))*nx*nz
uy = -(Delta_z/ ((1-nz*nz)))*ny*nz
u = np.array([ux, uy])

In [ ]:
cov_u = np.cov(u)
mean_u = np.mean(u)
cov_y = np.cov(y)
mean_y = np.mean(y, axis=1)
mean_e = mean_y
cov_e = cov_y - cov_u #- np.outer( mean_y, mean_y )

print('u cov : ', cov_u)
print('u mean : ', np.mean(u))
print('')
print('y mean : ', mean_y)
print('y cov : ', cov_y)
print('')
print('error mean : ', mean_e)
print('error cov : ', cov_e)
print('')
print('error std x : ', np.sqrt(cov_e[0,0]))
print('error std y : ', np.sqrt(cov_e[1,1]))


In [ ]:
ex, ey = np.random.multivariate_normal(mean_e, cov_e, size=1600000).T
bins=np.linspace(-6,6,32)
Xb, Yb = np.meshgrid(bins, bins, indexing='ij')
h,bx,by = np.histogram2d(ex, ey, bins=bins)
Xb, Yb = np.meshgrid((bx[0:-1]+bx[1:])/2., (by[0:-1]+by[1:])/2., indexing='ij')
plt.pcolormesh(Xb, Yb, h)
plt.xlabel('ex')
plt.ylabel('ey')
plt.show()


In [ ]:
fig,ax = plt.subplots(1,1)
ax.grid(True)
bins_fine = np.linspace(-np.max(np.abs(ux))-1, np.max(np.abs(ux))+1, 600)
bins_rough = np.linspace(-np.max(np.abs(ux))-1, np.max(np.abs(ux))+1, 30)

h, b = np.histogram(ux,  bins=bins_fine, density=True)
bc = (b[0:-1]+b[1:])/2.
ax.plot(bc, h, 'k-', label='$u_x$',)

h, b = np.histogram(y[0],  bins=bins_rough, density=True)
bc = (b[0:-1]+b[1:])/2.
ax.plot(bc, h, 'k^--', label='$y_x$',)

h, b = np.histogram(y[1],  bins=bins_rough, density=True)
bc = (b[0:-1]+b[1:])/2.
ax.plot(bc, h, 'ko--', label='$y_y$',)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlabel(r'GB displacement [$\mu$m]', fontsize=16)
ax.set_ylabel(r'Likelihood', fontsize=16)
ax.set_ylim([0, .62])
ax.set_xlim([-1.5*7, 1.5*7])
ax.legend()
if 1: fig.savefig('gallery/y_dist.png', dpi=600, pad_inches=0.000001, bbox_inches='tight')

plt.show()

fig,ax = plt.subplots(1,1)
ax.grid(True)
xxe = np.linspace(-7.5, 8.5, 100)
e = (1/np.sqrt(2*np.pi*cov_e[0,0]))*np.exp( -0.5* (xxe-mean_e[0])**2 / cov_e[0,0] )
ax.plot(xxe, e, 'k-', label='$e_x$',)
e = (1/np.sqrt(2*np.pi*cov_e[1,1]))*np.exp( -0.5* (xxe-mean_e[1])**2 / cov_e[1,1] )
ax.plot(xxe, e, 'k--', label='$e_y$',)
ax.legend()
ax.set_xlabel(r'Estimated error [$\mu$m]', fontsize=16)
ax.set_ylabel(r'Likelihood', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
if 1: fig.savefig('gallery/error_func.png', dpi=600, pad_inches=0.000001, bbox_inches='tight')
plt.show()